In [1]:
import sys
!{sys.executable} -m pip install langchain
!{sys.executable} -m pip install openai
!{sys.executable} -m pip install pandas

In [2]:
from langchain.prompts import (
    PromptTemplate,
    FewShotPromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

#import openai
#from langchain import *
#from pydantic import *
import os 
import pandas as pd 

with open("secret.txt") as f:
    key=f.read()
os.environ['OPENAI_API_KEY']=key[:-1]


In [6]:


example_formatter = """sentence: {sentence}
Tag: {tag}"""

example_prompt = PromptTemplate(
    input_variables=["sentence", "tag"],
    template=example_formatter
)

suffix = "sentence: {sentence}\nTag:"

body_prefix = """Classify the following as one of:

1. body: if the description is about body parts
2. not body: if the description does not mention body parts """

position_prefix = """Classify the following as one of:

1. position: if the description mentions positions or facing directions
2. not position: if the description does not mention positions or facing directions """

shape_prefix = """Classify the following as one of:

1. shape: if the description references geometric shapes
2. not shape: if the description does not reference geometric shapes"""

abstract_prefix = """Classify the following as one of:

1. abstract: if the description is in terms of a type of person or creature
2. not abstract: if the description is concrete and not metaphorical """


def make_chat_template(instruction):
    system_message = SystemMessage(content=instruction)
    messages = []
    #for example in examples:
    #   messages.append(HumanMessage(content=example["sentence"]))
    #   messages.append(AIMessage(content=example["tag"]))
    messages.append(HumanMessagePromptTemplate.from_template("{test_sentence}"))

    return ChatPromptTemplate.from_messages([system_message, *messages])

body_chat = make_chat_template(body_prefix)
position_chat = make_chat_template(position_prefix)
shape_chat = make_chat_template(shape_prefix)
abstract_chat = make_chat_template(abstract_prefix)

    

In [7]:



def tag_data_with_langchain(df, model, prompt): 
  
  res = []
  #for index in range(100):
  for index, _ in df.iterrows():
    print(index)
    #print(model)
    #print(prompt)
    chain=LLMChain(llm=model,prompt=prompt)
    output=chain.run(df["gpt_out"][index])
    print(df["gpt_out"][index])
    #string_output=output.content
    print(output)
    res.append(output)
  return res





In [10]:
model_name = "gpt-3.5-turbo"
#model_name="gpt-4"
temperature = 0
llm = ChatOpenAI(model_name=model_name, max_retries=15, temperature=0)

chain=LLMChain
input =pd.read_csv("data/test/sample_to_tag.csv")

test_input=input
#print(test_input)

body = tag_data_with_langchain(test_input, llm, body_chat)
position = tag_data_with_langchain(test_input, llm, position_chat)
shape= tag_data_with_langchain(test_input, llm, shape_chat)
abstract = tag_data_with_langchain(test_input, llm, abstract_chat)

output=test_input
output['body']=body
output['position']=position
output['shape']=shape
output['abstract']=abstract

output.to_csv("data/test/outsample_tag1.csv")


0
facing left
not body
1
head to the right
body
2
over his back
not body
3
leg out to the left
body
4
his arms are level with his head
body
5
his head comes past the right side of his body by a bit
body
6
the only one with a leg out to the left
body
7
the persons legs are facing the left
body
8
his head which is shaped almost like the box is facing to the right
body
9
walking to the left
not body
10
a bit like the thriller dance
not body
11
a man walking to the left
not body
12
with a square backpack on
not body
13
triangle on his face
body
14
one leg on floor to the left
body
15
one leg left is slightly raised
body
16
leg lifted to the left
body
17
thriller dance
not body
18
thriller
not body
19
a person dancing
not body
20
one foot on the floor
body
21
another foot is raised
body
22
tilted back
not body
23
made from a square shape
not body
24
4 sided shape
not body
25
the last one
not body
26
we havent selected
not body
27
the one with 2 heads almost
not body
28
dancing man
not body


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


kicking to the left
not shape
105
head falling off
not shape
106
two legs
not shape
107
1 on the floor
not shape
108
the other kicking to the left
not shape
109
head is off centre
not shape
110
both legs showing
not shape
111
one kicking left
not shape
112
diamond coming out of right side of head
not shape
113
standing facing left
not shape
114
on one foot
not shape
115
head falling off
not shape
116
a person kicking one foot in front of them
not shape
117
Their head looks like it has a beak
not shape
118
a box over their right shoulder
not shape
119
looking like a person carrying another person over their shoulder
not shape
120
michael jackson thriller dance
not shape
121
facing left
not shape
122
one leg
not shape
123
foot on the ground
not shape
124
the other lifting up
not shape
125
lifting forward
not shape
126
man facing left
not shape
127
kicking with diamond on right side
not shape
128
head
not shape
129
thriller dance
not shape
130
thriller
not shape
131
one leg planted
not sh

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


ice skating
not shape
251
walking zombie
not shape
252
square tilted back
shape
253
figure facing the left
not shape
254
one leg is down
not shape
255
one leg is up
not shape
256
one leg up on the left
not shape
257
one on the ground
not shape
258
dancing birdman
not shape
259
facing left
not shape
260
big triangle for a head
shape
261
square balanced over right shoulder
not shape
262
with one leg up
not shape
263
Bird man
not shape
264
Square on right
shape
265
balancing on corner
not shape
266
Left foot off the ground
not shape
267
right foot on
not shape
268
bird man dancing
not shape
269
bird man
not shape
270
Square head
shape
271
to the top right
not shape
272
Two legs on the left
not shape
273
one is lifted up
not shape
274
falling
not shape
275
the head
not shape
276
falling head
not shape
277
both legs are on the left
not shape
278
square head
shape
279
"looks like its falling off"
not shape
280
code name was FALLING
not shape
281
falling head
not shape
282
two feet facing lef

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


facing left
not abstract
71
person kicking one leg out to the left
not abstract
72
leg kicking to the left
not abstract
73
person kicking leg forward to the left
not abstract
74
someone stood up
not abstract
75
feet both pointing left
not abstract
76
a square
not abstract
77
a trapezium
not abstract
78
at the top
not abstract
79
kind of with arms outstretched
not abstract
80
on the left
not abstract
81
one leg in the air
not abstract
82
with one foot in the air
not abstract
83
person standing with diamond head leaning back
not abstract
84
arms stretched to the left
not abstract
85
one foot up
not abstract
86
facing left
not abstract
87
drunken zombie
1. abstract
88
a horse about to gallop
not abstract
89
Facing the left
not abstract
90
both feet are visible
not abstract
91
One foot on the floor
not abstract
92
one raised up to the left
not abstract
93
drunken zombie pose
not abstract
94
facing left
not abstract
95
with head tipping back towards the right
not abstract
96
one leg in the 